In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from utils import prepare_df, group_cases_by_trace

In [11]:
# load your DataFrame and softmax list
result = prepare_df('50salads')
if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# group by trace and inspect
trace_groups = group_cases_by_trace(df)
trace_groups

,case_list,trace_length
0,"[0, 1, 2, 3]",5687
1,"[32, 33, 34, 35]",6186
2,"[36, 37, 38, 39]",5840
3,"[28, 29, 30, 31]",5261
4,"[4, 5, 6, 7]",6208
5,"[16, 17, 18, 19]",6293
6,"[24, 25, 26, 27]",6046
7,"[8, 9, 10, 11]",6584
8,"[12, 13, 14, 15]",5558
9,"[20, 21, 22, 23]",5792


In [13]:
from data_processing import sample_and_write_collapsed_traces

sample_and_write_collapsed_traces(df, trace_groups, 10, 'sampled_traces.txt', random_seed=42)

Sampled 10 trace IDs: ['12', '32', '18', '1', '9', '37', '20', '4', '31', '24']
